# Test Attractor Dynamics with LIF Neurons

In [11]:
# Imports and funcs
%matplotlib widget
from brian2 import *
import numpy as np
import scipy.sparse as sp
import excitation_schedule as es
import brian_weight_submatrix as bws
import diagonal_sums as ds
import time

In [44]:
start_scope()
# The following line suppresses a warning about order of executions in the abstract code: "v_post = clip(v_post + w, v_gaba, 0)"
# As far as I can tell, the warning is due to the inability of the OOE checker to deal with the "clip()" function, and is ok in this case
BrianLogger.suppress_hierarchy('brian2.codegen')

seed(seed=1)

################## INDEPENDENT PARAMETERS ################
# Node parameters
t_mem = 10*ms
t_adapt_e = 100*ms
t_adapt_i = 100*ms
t_refract_e = 6.3*ms       # 6.3 ± 1.7 Raastad 2003
t_refract_std_e = 1.7*ms
t_refract_i = 5*ms # jittered with std=1 below
v_rest = -70*mV
v_adapt_step_e = 6*mV # 4
v_adapt_step_i = 5*mV # 4
v_thresh = -50*mV
v_reset = -65*mV
v_gaba = -80*mV

# Network parameters
net_size = 1.0*cm # circumference of toroidal network - brain width: mouse=1cm, human=15cm
nxe = 100 #100
nye = 100 #100
nxi = 40 #40
nyi = 40 #40
p0ee = 0.8
lradee = 30 # in units of neuron spacing
p0ei = 0.4
lradei = 2 # in units of neuron spacing
p0ie = 0.3
lradie = 5 # [5] in units of neuron spacing
wbase_ee = 0.0
wrange_ee = 0.2
wbase_ei = 3.0
wrange_ei = 6.0
wbase_ie = -4.0
wrange_ie = -5.0
delay_ee = True
dendritic_delay_min = 2.0 * ms # 3ms from Jarsky NatNeuro 2005
dendritic_delay_range = 2.0 * ms
conduction_velocity = 5.0 * meter / second

# STDP parameters
t_stdppre = 20.0 * ms
t_stdppost = 20.0 * ms
dApre = 0.128 * mV # 0.114@1cm 0.126@15cm
STDP_neg_pos_ratio = 1.05 # 1.05 from Song, Miller, Abbott 2000
wmax = 5.0 * mV # Need about 20mV to trigger a spike
STDP_delayed = True

# Stimulus parameters
attractor_size = 500
attractor_type = 'random' # 'random' or 'circular'
stim_freq = 8.0 # Hz
stim_duty = 0.25 # 0.25
stim_time = 1100 * ms # 1100
stim_dt = 1 * ms
stim_rate = 200 * Hz # 200
stim_ramp_on = 0.1
stim_ramp_off = 0.1
restim_delay = 0 * ms # 200
restim_time = 10 * ms # 10
restim_runtime = 140 * ms # 200
restim_fraction = 0.25 # 0.2
restim_density = 1.0 # 1.0
restim_N = 2

# Timing parameters
defaultclock.dt = 0.01*ms

################## CALCULATED PARAMETERS ################
# Network geometry
Ne = nxe * nye
Ni = nxi * nyi
dxe = net_size / nxe
dye = net_size / nye
dxi = net_size / nxi
dyi = net_size / nyi

# STDP
dApost = -dApre * t_stdppre / t_stdppost * STDP_neg_pos_ratio

########################## ATTRACTOR ########################
# Set attractor nodes
attractor_nodes = np.random.choice(Ne,attractor_size,replace=False)
if attractor_type == 'circular':
    attractor_nodes = np.sort(attractor_nodes)
attractor_index = {e:i for i,e in enumerate(attractor_nodes)} # lookup dictionary for getting attractor index from node number

########################## STIMULUS ########################
# Set stimulus
ex = es.excitation_schedule(attractor_size,'circular',stim_dt/ms,stim_time/ms,stim_duty,stim_freq,ramp_on=stim_ramp_on,ramp_off=stim_ramp_off)
ex = np.vstack((ex,np.zeros((int(restim_delay/stim_dt),attractor_size))))
stimulus_teach = TimedArray(ex*stim_rate,dt=stim_dt)

restim = np.zeros((int(restim_runtime/stim_dt),attractor_size))
restim_nodes = np.random.choice(int(attractor_size * restim_fraction),int(attractor_size * restim_fraction * restim_density))
restim[:int(restim_time/stim_dt),restim_nodes] = 1.
stimulus_test = TimedArray(restim*stim_rate,dt=stim_dt)

########################## MODEL ########################
# E nodes decay towards v_rest-v_adapt, and v_adapt decays towards 0
eqs_e = '''
dv/dt = (v_rest-v-v_adapt)/t_mem : volt (unless refractory)
dv_adapt/dt = (-v_adapt)/t_adapt_e : volt
x : meter
y : meter
net_size : meter
refract : second
'''
reset_e = '''
v = v_reset
v_adapt += v_adapt_step_e
'''

# I nodes decay towards v_rest-v_adapt, and v_adapt decays towards 0
eqs_i = '''
dv/dt = (v_rest-v-v_adapt)/t_mem : volt (unless refractory)
dv_adapt/dt = (-v_adapt)/t_adapt_i : volt
x : meter
y : meter
net_size : meter
refract : second
'''
reset_i = '''
v = v_reset
v_adapt += v_adapt_step_i
'''

# STDP Synapse equations
STDP_eqs = '''
w : volt
dApre/dt = -Apre / t_stdppre : volt (event-driven)
dApost/dt = -Apost / t_stdppost : volt (event-driven)
'''
STDP_onpre = '''
v_post += w
Apre += dApre
'''
STDP_onpost = '''
Apost += dApost
'''
if STDP_delayed:        # Accumulate prospective changes in dw to apply later
    STDP_eqs = STDP_eqs + 'dw : volt\n'
    STDP_onpre = STDP_onpre + 'dw += Apost\n'
    STDP_onpost = STDP_onpost + 'dw += Apre\n'
else:                   # Apply weight changes immediately
    STDP_onpre = STDP_onpre + 'w = clip(w + Apost, 0, wmax)\n'
    STDP_onpost = STDP_onpost + 'w = clip(w + Apre, 0, wmax)\n'

# Lorentz connection probability (mod (%) stuff makes it toroidal)
pLorentz = 'p0 / (1 + (((x_pre-x_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2 \
                     + ((y_pre-y_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2) / (lrad)**2)'

# Calculated conduction delay (mod (%) stuff makes it toroidal)
conduction_delay = 'dendritic_delay_min + rand() * dendritic_delay_range \
                        + sqrt( (((x_pre-x_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2 \
                        + ((y_pre-y_post + 1.5*net_size_post) % net_size_post - 0.5*net_size_post)**2) ) / conduction_velocity'

###################### CONSTRUCT NETWORK ####################
# Generate nodes
PGteach = PoissonGroup(attractor_size, rates='stimulus_teach(t,i)',name='PGteach')
PGtest  = PoissonGroup(attractor_size, rates='stimulus_test(t,i)',name='PGtest')
E = NeuronGroup(nxe * nye, eqs_e, threshold='v>v_thresh', reset=reset_e, refractory='refract', method='euler', name='E')
I = NeuronGroup(nxi * nyi, eqs_i, threshold='v>v_thresh', reset=reset_i, refractory='refract', method='euler', name='I')

# set neuron locations (zero indexed, so net is in quadrant 1)
E.x = '(i % nxe) * dxe'
E.y = '(i // nxe) * dye'
I.x = '(i % nxi) * dxi'
I.y = '(i // nxi) * dyi'
E.net_size = net_size
I.net_size = net_size

# Set refractory periods with jitter
E.refract = t_refract_e + t_refract_std_e * randn(len(E))
I.refract = t_refract_i * (1.0 + 0.2 * randn(len(I)))

# Connect Poisson Groups to E
SPGteach = Synapses(PGteach, E, on_pre='v_post += 50*mV') # 50 guarantees firing of post
SPGteach.connect(i=np.arange(attractor_size),j=attractor_nodes)
SPGtest  = Synapses(PGtest, E, on_pre='v_post += 50*mV') # 50 guarantees firing of post
SPGtest.connect(i=np.arange(attractor_size),j=attractor_nodes)

# Connect EE, EI, IE
# No self connections in EE network
SEE = Synapses(E, E, STDP_eqs, on_pre=STDP_onpre, on_post=STDP_onpost, name='EE')
SEE.variables.add_constant('p0',p0ee)
SEE.variables.add_constant('lrad',lradee * net_size / nxe) # Convert from neuron spacing units to meters here
SEE.connect(condition = 'i != j', p=pLorentz)
SEE.w = '(wbase_ee + wrange_ee*rand())*mV'
if delay_ee == True:
    SEE.delay = conduction_delay

SEI = Synapses(E, I, 'w : volt', on_pre='v_post += w', name='EI')
SEI.variables.add_constant('p0',p0ei)
SEI.variables.add_constant('lrad',lradei * net_size / nxi) # Convert from neuron spacing units to meters here
SEI.connect(p=pLorentz)
SEI.w = '(wbase_ei + wrange_ei*rand())*mV'

SIE = Synapses(I, E, 'w : volt', on_pre='v_post = clip((v_post + w), v_gaba, 0)', name='IE')
SIE.variables.add_constant('p0',p0ie)
SIE.variables.add_constant('lrad',lradie * net_size / nxe) # Convert from neuron spacing units to meters here
SIE.connect(p=pLorentz)
SIE.w = '(wbase_ie + wrange_ie*rand())*mV'

########################## GO ########################
# Monitor stuff
MSE = SpikeMonitor(E,name='Espikemon')
MSI = SpikeMonitor(I,name='Ispikemon')
MVE = StateMonitor(E,'v',True,dt=max(defaultclock.dt,1.*ms),name='Estatemon')
MVI = StateMonitor(I,'v',True,dt=max(defaultclock.dt,1.*ms),name='Istatemon')

# Initialize nodes at v_rest
E.v = v_rest
I.v = v_rest
store('initial_state')

# Initialize records for each teach/test period
time_record = []
spike_record = []
train_record = []

# Train
PGtest.active = False # only teacher for now
run(stim_time + restim_delay )

time_record.append(MSE.t/ms)
spike_record.append(1*MSE.i)
train_record.append(MSE.spike_trains())
if STDP_delayed:
    dw0 = 1.0 * SEE.dw # Multiply by 1.0 to force copy

# Test
PGteach.active = False
PGtest.active = True
for irestim in range(restim_N):
    if STDP_delayed:
        restore('initial_state')
        SEE.w = clip(SEE.w + (irestim+1)*dw0,0,wmax)
        SEE.dw = 0
    if irestim==0:
        w1 = bws.brian_weight_submatrix(SEE,attractor_nodes,attractor_nodes)
    run(restim_runtime)
    time_record.append(MSE.t/ms)
    spike_record.append(1*MSE.i)
    train_record.append(MSE.spike_trains())

w2 = bws.brian_weight_submatrix(SEE,attractor_nodes,attractor_nodes)


In [70]:
figure(figsize=(12,8))
nbin = 40

period = (0,1,2)
freq = []
ttl = ['During Training','Threshold Learning Rate','2x Threshold']
for idx,per in enumerate(period):
    spikes_in_attractor = np.isin(spike_record[per],attractor_nodes)
    z = spike_record[per][spikes_in_attractor]
    spikes = np.ndarray(z.shape)
    for node in attractor_index:
        spikes[z==node] = attractor_index[node]
    ts = time_record[per][spikes_in_attractor]
    points_in_window = (ts>(np.amin(ts)+20)) & (ts<(np.amax(ts)-20))
    tfit = ts[points_in_window]
    spikesfit = spikes[points_in_window]
    real,timeline = np.histogram(tfit,bins=nbin,weights=np.cos(2.0*np.pi*spikesfit/attractor_size))
    imag,timeline = np.histogram(tfit,bins=nbin,weights=np.sin(2.0*np.pi*spikesfit/attractor_size))
    timeline = 0.5 * (timeline[:-1]+timeline[1:])
    cplx = real + 1.j * imag
    phase = np.unwrap(np.angle(cplx))
    p = np.polyfit(timeline,phase,1,w=np.absolute(cplx))
    freq.append(1000. * p[0] / (2.*np.pi))
    subplot(2,3,idx+1)
    plot(ts,spikes,'k.',markersize=1)
    xlabel('Time (ms)')
    ylabel('Neuron Index')
    # plot(timeline,(attractor_size / (2.*np.pi) * (p[1] + timeline * p[0])) % attractor_size,'ro-')
    # title('f='+str(freq[idx])+'Hz')
    title(ttl[idx]) 

subplot(2,3,4)
imshow(w1,vmax=2)
xlabel('Neuron Index')
ylabel('Neuron Index')

subplot(2,3,5)
conduction_delay = 3.64e-3
k1 = -conduction_delay * freq[1] * attractor_size
k2 = -conduction_delay * freq[2] * attractor_size
plot([k1,k1],[0,150],'b')
plot([k2,k2],[0,150],'r')
plot(np.arange(attractor_size)-attractor_size/2,ds.diagonal_sums(w1),'b',label='Threshold Learning Rate')
plot(np.arange(attractor_size)-attractor_size/2,ds.diagonal_sums(w2),'r',label='2x Threshold')
xlabel('Diagonal Number (k)')
ylabel('Sum of Diagonal')
axis([-280,280,-30,800])
legend()

tight_layout()
savefig('FIG1.pdf')

WARNING    <ipython-input-70-70408e00eb66>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure(figsize=(12,8))
 [py.warnings]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
print(k1,k2)

-75.65331964439238 -132.78269215411342
